In [43]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import pygame
import sys

In [127]:
# Genrate Medications
meds_df = pd.DataFrame({
    'Medicine': [],
    'Lifespan': []
})
max_meds = 10
lifespan_mean = 25
lifespan_std = 10

LIFESPAN_MIN = 5

medicine = set(['{:04x}'.format(np.random.randint(0, 0xFFFF)) for _ in range(max_meds)])
# ensures there are max_meds unique values
while len(medicine) < max_meds: medicine.add('{:04x}'.format(np.random.randint(0, 0xFFFF)))
medicine = list(medicine)

lifespan = np.random.normal(lifespan_mean, lifespan_std, max_meds)
lifespan = np.round(np.maximum(lifespan, LIFESPAN_MIN))

meds_df['Medicine'] = medicine
meds_df['Lifespan'] = lifespan

meds_df.head()


,Medicine,Lifespan
0,ce87,25.0
1,719f,21.0
2,bb31,6.0
3,7dcf,23.0
4,413e,10.0


In [128]:
# Generate Clients
client_count = 100
MAXLAT = 256
MAXLONG = 256
MAX_DOSAGE = 3

latitudes = np.random.randint(0, MAXLAT, client_count)
longitudes = np.random.randint(0, MAXLONG, client_count)

cs_df = pd.DataFrame({
    'Client': ['Client {:03d}'.format(i) for i in range(client_count)],
    'Lat': latitudes,
    'Long': longitudes,
    'Medicine': np.random.choice(medicine, client_count),
    'Dosage': np.random.randint(1, MAX_DOSAGE, client_count)
})
# cs_df.index.name = 'Client'
cs_df.head()

,Client,Lat,Long,Medicine,Dosage
0,Client 000,139,154,ddd4,1
1,Client 001,200,181,ddd4,1
2,Client 002,21,98,bb31,1
3,Client 003,41,102,628a,1
4,Client 004,144,6,413e,1


In [129]:
# Generate Pharmacies
pharmacy_count = 10
pharmacy_names = [f'pharm{i}' for i in range(pharmacy_count)]

latitudes = np.random.randint(0, MAXLAT, pharmacy_count)
longitudes = np.random.randint(0, MAXLONG, pharmacy_count)

rx_df = pd.DataFrame({
    'Name': pharmacy_names,
    'Lat': latitudes,
    'Long': longitudes
})
rx_df

,Name,Lat,Long
0,pharm0,158,11
1,pharm1,49,243
2,pharm2,96,43
3,pharm3,162,85
4,pharm4,118,134
5,pharm5,226,16
6,pharm6,18,220
7,pharm7,91,25
8,pharm8,239,5
9,pharm9,103,36


In [130]:
# Assign Customers to Pharmacies
# client are assigned to closest pharm

# distance between each client and each pharm
client_coords = cs_df[['Lat', 'Long']].values
pharm = np.column_stack((latitudes[:pharmacy_count], longitudes[:pharmacy_count]))

distances = np.sqrt(
    (
        (client_coords[:, np.newaxis, :] - pharm[np.newaxis, :, :]) ** 2
    ).sum(axis=2))


# Assign each customer to the closest pharmacy
closest_pharmacies = np.argmin(distances, axis=1)
cs_df['Assigned_Pharmacy'] = closest_pharmacies

cs_df.head()


,Client,Lat,Long,Medicine,Dosage,Assigned_Pharmacy
0,Client 000,139,154,ddd4,1,4
1,Client 001,200,181,ddd4,1,4
2,Client 002,21,98,bb31,1,2
3,Client 003,41,102,628a,1,2
4,Client 004,144,6,413e,1,0


In [131]:
# Sort by assigned pharmacy
total_cs_df = cs_df[['Assigned_Pharmacy','Medicine','Dosage']].groupby(['Assigned_Pharmacy','Medicine']).sum().reset_index().set_index('Assigned_Pharmacy')
total_cs_df.head()

,Medicine,Dosage
Assigned_Pharmacy,,
0,413e,3
0,75cd,2
0,7dcf,2
0,98cf,2
0,ce87,1


In [132]:
merged_df = pd.merge(rx_df, total_cs_df, right_index=True, left_index=True, how='left')
merged_df.sample(5)

,Name,Lat,Long,Medicine,Dosage
7,pharm7,91,25,719f,3
4,pharm4,118,134,7dcf,6
7,pharm7,91,25,75cd,4
5,pharm5,226,16,413e,1
5,pharm5,226,16,ce87,1


In [133]:
# Assign each pharmacy a list of medications that are required
from collections import Counter

# use assigned_pharmacy in cs_df to get the pharmacy name, and then get the medicine
# turn that into a counter and then into a dataframe into demand
def calculate_demand(group):
    return Counter({medicine: group.loc[group['Medicine'] == medicine, 'Dosage'].sum() for medicine in group['Medicine'].unique()})

# Apply the function to group by pharmacy ('Name') and calculate demand
demand_agg = merged_df.groupby('Name').apply(calculate_demand)

# Assign the aggregated demand back to the original DataFrame
pharmacy_df = merged_df.drop_duplicates(subset='Name').set_index('Name')
pharmacy_df['Demand'] = demand_agg
pharmacy_df = pharmacy_df.drop(columns=['Medicine', 'Dosage'])

# Display the result
pharmacy_df.reset_index()


/tmp/ipykernel_48747/2529929852.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  demand_agg = merged_df.groupby('Name').apply(calculate_demand)


,Name,Lat,Long,Demand
0,pharm0,158,11,"{'413e': 3, '75cd': 2, '7dcf': 2, '98cf': 2, '..."
1,pharm1,49,243,"{'628a': 4, '7dcf': 1, '98cf': 3, 'bb31': 1, '..."
2,pharm2,96,43,"{'413e': 2, '628a': 1, '719f': 2, '75cd': 1, '..."
3,pharm3,162,85,"{'413e': 1, '628a': 4, '719f': 3, '75cd': 3, '..."
4,pharm4,118,134,"{'413e': 6, '628a': 7, '719f': 1, '75cd': 1, '..."
5,pharm5,226,16,"{'413e': 1, 'c171': 2, 'ce87': 1}"
6,pharm6,18,220,"{'413e': 3, '75cd': 2, '98cf': 1, 'bb31': 5, '..."
7,pharm7,91,25,"{'413e': 3, '628a': 1, '719f': 3, '75cd': 4, '..."
8,pharm8,239,5,{'c171': 2}
9,pharm9,103,36,"{'98cf': 2, 'ddd4': 2}"
